In [61]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep, strftime
from random import randint
import pandas as pd
from credentials import *
from config import *
import logging

logging.basicConfig(format='%(levelname)s [%(asctime)s] %(message)s', datefmt='%m/%d/%Y %r', level=logging.INFO)
logger = logging.getLogger()

print('Libraries read!')

Libraries read!


In [62]:
url = 'https://www.instagram.com/accounts/login/?source=auth_switcher'
chromedriver_path = '/home/juan/Desktop/insta/venv/chromedriver' # Change this to your own chromedriver path!
browser = webdriver.Chrome(executable_path=chromedriver_path)
browser.get(url)
sleep(2)

# cookie 
cookie_button = browser.find_element_by_xpath("/html/body/div[4]/div/div/button[1]")
cookie_button.click()

username = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

username.clear()
username.send_keys('juan_adenip')
password.clear()
password.send_keys('Juancho57*')
Login_button = WebDriverWait(browser, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

sleep(3)

# not now
not_now = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]'))).click()
not_now2 = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]'))).click()






In [63]:
# List of hashtags to go through
hashtag_list = ['beautifuldestinations', 'photobomb', 
                #'nature_photography', 'visitluxembourg', 'bestdestinations', 'luxembourg'
                ]

# List of comments to be randonmly chosen from
commentsList = ['Really cool!', 'Nice work ❤️', '👏👏', 'wow 👏', 'well said', 'so cool! 🙌', '❤️', 'lovely 😍', 'amazing 👌', 'amazing', 'wow ❤️', 'well said 👏', '👏👏👏👏', 'Nice!', 'nice 🔥', 'feels good 😍', 'Good job! 😊', '❤️🔥🔥😍', 'thoughtful 👌', 'thatz deep', 'intense 💯', '😊😊', 'i will take it as quote of the day','too much motivation 👌','very inspiring ❤️🔥']

# Number of posts to go through per hashtag
number_of_posts = 10

# Chance of commenting on photo
# i.e. chance_to_comment = 4 means a 1/4 chance
chance_to_comment = 2

# Time to wait in between processing instagram posts in seconds
# Enter lower and upper limit in randint()
wait_between_posts = randint(2, 10)

# Time to wait in between liking a post and commenting on it in seconds
# Enter lower and upper limit in randint()
wait_to_comment = randint(2, 9)

In [64]:
#prev_user_list = [] # if it's the first time you run it, use this line and comment the two below
#prev_user_list = pd.read_csv('20181203-224633_users_followed_list.csv', delimiter=',').iloc[:,1:2] # useful to build a user log
#prev_user_list = list(prev_user_list['0'])

likes = 0
comments = 0
follows = 0

for hashtag in hashtag_list:
    browser.implicitly_wait(30)
    browser.get(f'https://www.instagram.com/explore/tags/{hashtag}/')
    logger.info(f'Exploring #{hashtag}')
    sleep(randint(1,2))

    # Click first thumbnail to open
    browser.implicitly_wait(30)
    first_thumbnail = browser.find_element_by_xpath("//*[@id='react-root']/section/main/article/div[1]/div/div/div[1]/div[1]/a/div/div[2]")
    first_thumbnail.click()

    sleep(randint(1,2))

    # Go through x number of photos per hashtag
    for post in range(1,number_of_posts):

        try:
                
            # Follow
            browser.implicitly_wait(30)
            if browser.find_element_by_xpath('/html/body/div[6]/div[2]/div/article/header/div[2]/div[1]/div[2]/button').text == 'Follow':
                browser.find_element_by_xpath("/html/body/div[6]/div[2]/div/article/header/div[2]/div[1]/div[2]/button").click()            
            logger.info("Followed")
            follows += 1
                
            # Like
            browser.find_element_by_xpath('//*[@aria-label="Like"]').click()
            logger.info("Liked")
            likes += 1
                
            sleep(randint(2,4))

            # Comment
            try:
                browser.implicitly_wait(30)
                browser.find_element_by_class_name('Ypffh').click()
                # Random chance of commenting
                do_i_comment = randint(1,chance_to_comment)
                if do_i_comment == 1:

                    browser.implicitly_wait(30)
                    comment = browser.find_element_by_class_name('Ypffh')
                    comment.click()

                    sleep(wait_to_comment)

                    rand_comment_index = randint(0,len(comments_list))
                    comment.send_keys(comments_list[rand_comment_index])
                    comment.send_keys(Keys.ENTER)
                    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[type='submit']"))).click() # TEST
                    logger.info(f"Commented '{comments_list[rand_comment_index]}'")
                    comments += 1
                    sleep(wait_between_posts)

            except Exception:
                # Continue to next post if comments section is limited or turned off
                pass

        except Exception:
            # Already liked it, continue to next post
            logger.info('Already liked this photo previously')
            pass
            
        # Go to next post
        browser.implicitly_wait(30)
        browser.find_element_by_link_text('Next').click()
        logger.info('Getting next post')
        sleep(wait_between_posts)    

logger.info(f'Followed {follows} accounts')
logger.info(f'Liked {likes} posts')
logger.info(f'Commented on {comments} posts')



INFO [08/16/2021 04:30:12 PM] Exploring #beautifuldestinations
INFO [08/16/2021 04:30:15 PM] Followed
INFO [08/16/2021 04:30:15 PM] Liked
INFO [08/16/2021 04:30:18 PM] Getting next post
INFO [08/16/2021 04:30:28 PM] Followed
INFO [08/16/2021 04:30:28 PM] Liked
INFO [08/16/2021 04:30:31 PM] Getting next post
INFO [08/16/2021 04:30:41 PM] Followed
INFO [08/16/2021 04:30:41 PM] Liked
INFO [08/16/2021 04:30:43 PM] Getting next post
INFO [08/16/2021 04:30:53 PM] Already liked this photo previously


WebDriverException: Message: chrome not reachable
  (Session info: chrome=92.0.4515.131)


In [ ]:
browser.close()